In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import seaborn as sns
from scipy.stats import gaussian_kde
from matplotlib.patches import Polygon

warnings.filterwarnings("ignore")

In [ ]:
def scatter_with_gaussian_kde(ax, x, y, **kwargs):
    """
    Plots a scatter plot colored by gaussian kde estimates.

    Parameters:
    - ax (matplotlib ax): The ax on which to plot the scatter plot.
    - x (np.array): The x values to perform the gaussian kde and scattering on.
    - y (np.array): The y values to perform the gaussian kde and scattering on.
    - **kwargs: Additional keyword arguments to pass to the scatter function.

    Returns:
    - None
    """

    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    ax.scatter(x, y, c=z, **kwargs)

def import_quantification_data(base_directory="../data/IF/", celltype="P14"):
    """
    Imports IF quantification data from CSV files in the specified directory.

    Parameters:
    - base_directory (str): The directory containing the quantification data.
    - celltype (str): The cell type to filter the data by.

    Returns:
    - final_df (pd.DataFrame): A DataFrame containing the IF quantification data from the CSV files.
    """
    
    # Initialize an empty list to store DataFrames
    dataframes = []

    # Loop through subdirectories
    for experiment_folder in os.listdir(base_directory):
        experiment_path = os.path.join(base_directory, experiment_folder)

        if os.path.isdir(experiment_path):
            # Initialize an empty list to store DataFrames within the experiment
            experiment_dataframes = []

            # Loop through CSV files in the experiment folder
            for csv_file in os.listdir(experiment_path):
                if csv_file[-4:] == ".txt":
                    day = csv_file[:-4]
                    df = pd.read_csv(os.path.join(experiment_path, csv_file), sep="\t")
                    df["experiment"] = experiment_folder
                    df["day"] = day
                    df = df[df["Class"] == celltype]
                    experiment_dataframes.append(df)

            # Concatenate DataFrames within the experiment
            experiment_df = pd.concat(experiment_dataframes, ignore_index=True)
            dataframes.append(experiment_df)

    # Concatenate all experiment DataFrames into one final DataFrame
    final_df = pd.concat(dataframes, ignore_index=True)

    final_df["Exp_day"] = final_df["experiment"] + "_" + final_df["day"]

    # Now, final_df contains all the data from CSV files with added 'experiment' and 'day' columns
    final_df
    return final_df

def draw_gates(ax, gates, transformation, type="edge"):
    """
    Draws IMAP gates on a matplotlib ax.

    Parameters:
    - ax (matplotlib ax): The ax on which to draw the gates.
    - gates (dict): A dictionary containing the gates to draw.
    - transformation (function): A function to transform the x values of the gates.
    - type (str): The type of gate to draw. Can be "fill" or "edge".

    Returns:
    - None
    """

    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]

        if type == "fill":
            p = Polygon(points, facecolor=gates[gate]["fill"], edgecolor="none")
            ax.add_patch(p)
        elif type == "edge":
            p = Polygon(points, facecolor="none", edgecolor="#222222")
            ax.add_patch(p)

            ax.text(
                transformation(gates[gate]["label_position"]["x"]),
                gates[gate]["label_position"]["y"],
                gate,
                fontsize=6,
                color="#222222",
            )

In [ ]:
# Coordinates of the IMAP gates
gates = {
    "Top": {
        "edges": [
            [-5, 600],
            [35, 600],
            [35, 240],
            [12, 200],
            [-5, 240],
        ],
        "label_position": {"x": 0, "y": 600},
        "fill": "#3A9AB244",
    },
    "Crypt": {
        "edges": [
            [-5, 230],
            [12, 190],
            [35, 230],
            [16, -10],
            [-5, -10],
        ],
        "label_position": {"x": 0, "y": 0},
        "fill": "#F11B0044",
    },
    "Muscularis": {
        "edges": [
            [37, 230],
            [65, 120],
            [65, -10],
            [18, -10],
        ],
        "label_position": {"x": 70, "y": 20},
        "fill": "#BDC88155",
    },
}

In [ ]:
# import the IMAP quantification data
df = import_quantification_data()
df.head()

Plotting IMAPs for all experiments along the IF timecourse

In [ ]:
def ceildiv(a, b):
    return -(a // -b)


xmin, xmax = np.min(df["Distance to detection with Epithelial µm"]), np.max(
    df["Distance to detection with Epithelial µm"]
)

experiments = df["Exp_day"].unique()
experiments.sort()
ncols = 5
nrows = ceildiv(len(experiments), ncols)

fig = plt.figure(figsize=(3 * ncols, 3 * nrows), dpi=600)

df["epithel_transformed"] = df[
    "Distance to detection with Epithelial µm"
]  # We avoid plotting the transformed values here
xmin, xmax = np.min(df["epithel_transformed"]), np.max(df["epithel_transformed"])

for i, exp in enumerate(experiments):

    ax = fig.add_subplot(nrows, ncols, i + 1)
    ax.set_title(exp)

    df_sub = df[df["Exp_day"] == exp]

    df_sub = df_sub.dropna(
        subset=["epithel_transformed", "Distance to annotation with Base µm"], how="any"
    )
    if len(df_sub) == 0:
        continue

    sns.kdeplot(
        data=df_sub,
        x="epithel_transformed",
        y="Distance to annotation with Base µm",
        ax=ax,
        color="#444444",
        linewidth=0.5,
    )

    # Colored scatter plot
    scatter_with_gaussian_kde(
        ax=ax,
        x=df_sub["epithel_transformed"],
        y=df_sub["Distance to annotation with Base µm"],
        s=5,
    )

    ax.set_xlim(-10, 70)
    ax.set_ylim(-100, 700)

    # Draw gates
    draw_gates(ax, gates, lambda x: x, type="fill")

fig.tight_layout()